In [8]:
import pandas as pd
import numpy as np 
from scipy.stats import zscore
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [94]:
def raw_2080(rating):
    new = ((5 * rating) / (50/3))+20
    return new

def convert_2080_120(rating):
    new = math.ceil((round(((50/3) * (float(rating) - 20))/5,0))/10)
    return new

def convert_sbc_2080(rating):
    new = round((50 * (rating)/16.67)+20,0)
    return new

def convert_sbc_raw(rating):
    twenty = round((50 * (rating)/16.67)+20,0)
    raw = (((50/3) * (twenty - 20) )/ 5)
    return raw

def clean_players(df):
    df = df.copy()
    positions = {1: "P", 2: "C", 3: "1B", 4: "2B", 5: "3B", 6: "SS", 7: "LF", 8: "CF", 9: "RF", 10: "DH"}
    roles = {0: "BAT", 11: "SP", 12: "RP", 13: "CL"}
    bats = {1: "R", 2: "L", 3: "S"}
    throws = {1: "R", 2:"L"}
    ratings = ['speed', 'stealing', 'baserunning', 'greed', 'loyalty', 'work ethic','intelligence', 'leader']
    df = df[df['league_id'] == 155]
    df = df[['player_id','weight','height','position','role','bats','throws','running_ratings_speed', 'running_ratings_stealing','running_ratings_baserunning','personality_greed','personality_loyalty','personality_work_ethic','personality_intelligence','personality_leader']]
    df['position'].replace(positions, inplace=True)
    df['role'].replace(roles, inplace=True)
    df['bats'].replace(bats, inplace=True)
    df['throws'].replace(throws, inplace=True)
    df.rename(columns={"running_ratings_speed": "speed", "running_ratings_stealing": "stealing", "running_ratings_baserunning": "baserunning", "personality_greed": "greed", 'personality_loyalty': 'loyalty','personality_work_ethic': 'work ethic','personality_intelligence': "intelligence",'personality_leader': 'leader'}, inplace = True)
    df[ratings] = df[ratings].apply(raw_2080).round(0).astype(int)
    df['greed'] = np.where(df['greed'] < 45, "L", np.where(df['greed']< 59, "N", "H"))
    df['loyalty'] = np.where(df['loyalty'] < 45, "L", np.where(df['loyalty'] < 59, "N", "H"))
    df['work ethic'] = np.where(df['work ethic'] < 45, "L", np.where(df['work ethic'] < 59, "N", "H"))
    df['intelligence'] = np.where(df['intelligence'] < 45, "L", np.where(df['intelligence'] < 59, "N", "H"))
    df['leader'] = np.where(df['leader'] < 45, "L", np.where(df['leader'] < 59, "N", "H"))
    return df.set_index('player_id')

def clean_batters(df):
    df = df.copy()
    positions = {1: "P", 2: "C", 3: "1B", 4: "2B", 5: "3B", 6: "SS", 7: "LF", 8: "CF", 9: "RF", 10: "DH"}
    roles = {0: "BAT", 11: "SP", 12: "RP", 13: "CL"}
    bats = {1: "R", 2: "L", 3: "S"}
    throws = {1: "R", 2:"L"}
    ratings = ['contact', 'gap', 'power', 'eye', 'avoid_k', 'bunt','bunt_hit']
    df = df[df['league_id'] == 155]
    df = df[['player_id','position','role','batting_ratings_overall_contact', 'batting_ratings_overall_gap', 'batting_ratings_overall_power', 'batting_ratings_overall_eye', 'batting_ratings_overall_strikeouts', 'batting_ratings_misc_bunt', 'batting_ratings_misc_bunt_for_hit']]
    df['position'].replace(positions, inplace=True)
    df['role'].replace(roles, inplace=True)
    df.rename(columns={"batting_ratings_overall_contact": "contact", 
                        "batting_ratings_overall_gap": "gap", 
                        "batting_ratings_overall_eye": "eye", 
                        "batting_ratings_overall_strikeouts": "avoid_k", 
                        'batting_ratings_overall_power': 'power',
                        'batting_ratings_misc_bunt': 'bunt',
                        'batting_ratings_misc_bunt_for_hit': "bunt_hit",},
                         inplace = True)
    df[ratings] = df[ratings].apply(raw_2080).round(0).astype(int)
    df = df[df['role'] ==  'BAT'].drop('role', axis = 1)
    return df.set_index('player_id')

In [95]:
path = 'C:/Users/night/Documents/Out of the Park Developments/OOTP Baseball 22/saved_games/New Game.lg/import_export/csv/'
players = pd.read_csv(path + 'players.csv', low_memory = False)
batters = pd.read_csv(path + 'players_batting.csv', low_memory = False)

In [96]:
test = clean_players(players)
testtwo = clean_batters(batters)

In [97]:
test.head()

,weight,height,position,role,bats,throws,speed,stealing,baserunning,greed,loyalty,work ethic,intelligence,leader
player_id,,,,,,,,,,,,,,
2,200,183,1B,BAT,R,L,37,39,49,N,L,N,N,N
5,219,191,P,SP,L,L,30,25,35,L,N,H,N,N
15,183,188,SS,BAT,R,R,70,74,74,N,N,H,N,N
17,240,196,P,RP,L,L,26,29,31,N,N,L,H,N
18,199,188,P,SP,R,R,30,25,29,L,N,H,H,N


In [98]:
testtwo.head()

,position,contact,gap,power,eye,avoid_k,bunt,bunt_hit
player_id,,,,,,,,
2,1B,52,51,43,58,65,46,45
15,SS,71,64,53,51,58,55,56
21,LF,38,43,58,59,31,58,40
28,LF,54,49,57,58,44,52,31
44,C,35,40,51,41,37,51,26
